# Relatório tecnico-CNPq Universal

Reginaldo K Fukuchi, Feb 2023, reginaldo.fukuchi@ufabc.edu.br

Este notebook apresenta dados para prestacao de contas do projeto universal do CNPq

Fukuchi RK, Fukuchi CA and Duarte M (2017). A public data set of running biomechanics and the effects of running speed on lower extremity kinematics and kinetics. PeerJ Preprints.

In addition, it demonstrates plots of angles, moments, powers and ground reaction force curves displayed in the manuscript.

Some of the steps have been reduced to minimize clutter, but the user should be able to adapt this code to any given file structure.

In [1]:
# Prepare environment
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
# Import data
pathname = r'../data'

## Compare spreadsheets to find matching subjects
### Data from Figshare

In [3]:
fname    = os.path.join(pathname, 'RBDSinfo.txt') 
# Import data
data_p = pd.read_csv(fname, sep = '\t')
info = data_p.drop_duplicates(subset='Subject', inplace=False)
info.set_index('Subject', inplace=True)
info.head()

,FileName,Age,Height,Mass,Gender,Dominance,Level,Experience,SessionsPerWk,Treadmill,...,LThomas,ROber,LOber,RHIPABD,LHIPABD,RHIPEXT,LHIPEXT,RHIPER,LHIPER,RHIPIR
Subject,,,,,,,,,,,,,,,,,,,,,
1,RBDS001static.txt,22,181.0,62.00,M,R,Competitive,4,3,0,...,11,43,40,16.800000,21.575,16.575,21.675000,9.666667,7.250000,10.966667
2,RBDS002static.txt,38,183.0,80.00,M,R,Competitive,60,3,0,...,12,43,38,23.000000,28.025,23.700,21.500000,10.650000,9.600000,18.466667
3,RBDS003static.txt,34,177.0,65.40,M,L,Competitive,54,5,0,...,15,44,41,22.033333,24.300,23.325,19.050000,10.333333,10.733333,15.500000
4,RBDS004static.txt,33,168.0,57.05,M,R,Elite,216,4,1,...,11,34,32,18.300000,14.350,15.600,16.966667,9.100000,6.966667,12.066667
5,RBDS005static.txt,32,169.0,60.00,M,R,Competitive,36,5,1,...,4,33,29,18.833333,25.300,20.325,19.900000,12.200000,7.375000,10.700000


### Find the original index of the subjects
The data from Google forms used to interview the subjects. There are more than 28 subjects so we need to find their index

In [4]:
fname_q= os.path.join(pathname, 'RBDS_google_forms.csv') 
# Import data
data_q = pd.read_csv(fname_q, sep = ',', index_col = 'ID')
data_q.head()

,Timestamp,Primeiro nome,Último nome,E-mail,Data de nascimento,Altura (cm),Massa (kg),Gênero,Membro inferior dominante,Qual é o seu desempenho na corrida?,...,Caso tenha respondido sim na questão anterior,Data aproximada do inicio dos sintomas,"Quando você corre, os sintomas ocorrem...",Número do tênis de corrida,Marca do calçado,Modelo do calçado,Número de pares,Com que frequência você substitui os ses tênis?,Indique o nível de conforto do seu tênis de corrida atual,Você utiliza algum objeto dentro do tênis?
ID,,,,,,,,,,,,,,,,,,,,,
BMC0012,9/12/2016 15:31:28,Rodrigo,Lobo,rodrigo@loboassessoria.com.br,1/19/1983,168.0,57.05,Masculino,Direito,Elite,...,NaN,NaN,NaN,39,Asics,DS Racer,2,Menos de 6 meses,7,Nenhum
BMC0014,9/12/2016 9:21:17,José,Belarmino,jose-belarmino@hotmail.com,4/29/1977,177.8,78.30,Masculino,Direito,Elite,...,NaN,4/15/2015,Piora no início da corrida e melhora progressi...,42,Adidas,Adizero boost,1,Entre 7 meses e 1 ano,9,Nenhum
BMC0015,9/12/2016 9:26:10,Thiago,Lima,llthiago@hotmail.com,11/19/1985,182.9,66.70,Masculino,Direito,Competitivo,...,Inflamação na fíbula esquerda,12/15/2012,Piora no início da corrida e melhora progressi...,40,Adidas,Adizero boost,2,Menos de 6 meses,8,Nenhum
BMC0016,9/12/2016 9:30:58,Yan,Cardoso,Yan.hollerbach@gmail.com,8/20/1980,180.0,69.00,Masculino,Esquerdo,Competitivo,...,NaN,3/8/2016,Piora no início da corrida e melhora progressi...,41,Asics,Kayano/Noosa,4,Entre 7 meses e 1 ano,8,Nenhum
BMC0017,9/12/2016 9:40:59,Marcelo,Duarte,marcelo@,7/22/1965,173.2,63.15,Masculino,Direito,Recreacional competitivo,...,Lesão IQT E,9/8/2014,Somente no fim da corrida ou no dia seguinte,40,Asics,Norsa 10,1,Menos de 6 meses,6,Nenhum


#### Try using heigh and mass

In [5]:
df_rba = data_q[['Altura (cm)','Massa (kg)']] # Google forms
df_figshare = info[['Height','Mass']] # RBDS Figshare

In [6]:
ids = []
for i in range(df_figshare.shape[0]):
    df_out = df_rba.loc[(df_rba['Altura (cm)']==df_figshare['Height'].values[i]) & (df_rba['Massa (kg)']==df_figshare['Mass'].values[i])]
    if df_out.empty:
        print('Subject '+ str(df_figshare.index.values[i]) + ' not found!')
    else:
        ids.append(df_out.index.tolist())

Subject 1 not found!
Subject 2 not found!
Subject 3 not found!
Subject 5 not found!
Subject 23 not found!


In [7]:
len(ids)

23

In [8]:
ids

[['BMC0012'],
 ['BMC0014'],
 ['BMC0015'],
 ['BMC0016'],
 ['BMC0017'],
 ['BMC0018'],
 ['BMC0020'],
 ['BMC0023'],
 ['BMC0024'],
 ['BMC0025'],
 ['BMC0026'],
 ['BMC0027'],
 ['BMC0028'],
 ['BMC0029'],
 ['BMC0030'],
 ['BMC0032'],
 ['BMC0033'],
 ['BMC0034'],
 ['BMC0036'],
 ['BMC0037'],
 ['BMC0038'],
 ['BMC0039'],
 ['BMC0040']]

# PENDING
* Find who is who. 
Find the subjects in Figshare RBDSinfo.txt with the corresponding ID in RBA spreadsheet